In [8]:
import requests
import pandas as pd

import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [15]:
base_url = "https://base-sepolia.blockscout.com/api/v2/addresses/0xa4e91145fd2370eca42186b9614ae3df398832cd/internal-transactions?filter=to%20%7C%20from"
op_url = "https://optimism-sepolia.blockscout.com/api/v2/addresses/0x3e6f168587f9721a31f2fa1a560e6ab36d3b8c69/internal-transactions?filter=to%20%7C%20from"
node_url = "https://optimism-sepolia.blockscout.com/api/v2/addresses/0x35686511f74190b39683f4eb6a35231674a580b1/internal-transactions?filter=to%20%7C%20from"

# Make the GET request for base
base_response = requests.get(base_url)
# Make the GET request for op
op_response = requests.get(op_url)

# Make the GET request for node
node_response = requests.get(node_url)

# Function to extract success and timestamp from the JSON data
def extract_success_timestamp(data):
    items = data.get('items', [])
    records = []
    for item in items:
        success = item.get('success', None)
        timestamp = item.get('timestamp', None)
        records.append({'success': success, 'timestamp': timestamp})
    return records

# Helper function to process the response
def process_response(response):
    if response.status_code == 200:
        try:
            data = response.json()  # Attempt to parse JSON
            return extract_success_timestamp(data)
        except requests.exceptions.JSONDecodeError:
            print(f"Error: Response is not in JSON format. Content: {response.text}")
            return None
    else:
        print(f"Error: Received status code {response.status_code}. Content: {response.text}")
        return None

# Extract data from base response
base_records = process_response(base_response)

# Extract data from op response
op_records = process_response(op_response)

# Extract data from node response


In [16]:
node_records = process_response(node_response)

In [17]:
base_consumer = pd.DataFrame(base_records)
op_consumer = pd.DataFrame(op_records)
node = pd.DataFrame(node_records)

In [20]:
node_response.json()

{'items': [], 'next_page_params': None}

In [11]:
base_consumer['call_success'] = base_consumer['success'].apply(lambda x: 1 if x else 0)
base_consumer['call_fail'] = base_consumer['success'].apply(lambda x: 0 if x else 1)

# Drop the original 'success' column if no longer needed
base_consumer = base_consumer.drop(columns=['success'])

print(base_consumer)

                      timestamp  call_success  call_fail
0   2024-08-09T04:13:32.000000Z             1          0
1   2024-08-09T04:13:20.000000Z             1          0
2   2024-08-08T06:03:16.000000Z             1          0
3   2024-08-08T06:03:06.000000Z             1          0
4   2024-08-08T05:55:04.000000Z             1          0
5   2024-08-08T05:54:54.000000Z             1          0
6   2024-08-08T05:10:04.000000Z             1          0
7   2024-08-08T05:09:54.000000Z             1          0
8   2024-08-08T05:00:00.000000Z             1          0
9   2024-08-08T04:59:42.000000Z             1          0
10  2024-08-08T03:43:52.000000Z             1          0


In [12]:
op_consumer['call_success'] = op_consumer['success'].apply(lambda x: 1 if x else 0)
op_consumer['call_fail'] = op_consumer['success'].apply(lambda x: 0 if x else 1)

# Drop the original 'success' column if no longer needed
op_consumer = op_consumer.drop(columns=['success'])

In [14]:
node

""


In [13]:
node['call_success'] = node['success'].apply(lambda x: 1 if x else 0)
node['call_fail'] = node['success'].apply(lambda x: 0 if x else 1)

# Drop the original 'success' column if no longer needed
node = node.drop(columns=['success'])

KeyError: 'success'

In [ ]:
base_consumer['timestamp'] = pd.to_datetime(base_consumer['timestamp'])
op_consumer['timestamp'] = pd.to_datetime(op_consumer['timestamp'])
node['timestamp'] = pd.to_datetime(node['timestamp'])

In [ ]:
base_consumer['hourly'] = base_consumer['timestamp'].dt.floor('H')

# Step 2: Group by the hourly timestamp and sum the success counts
hourly_aggregated = base_consumer.groupby('hourly')[['call_success','call_fail']].sum().reset_index()

# Display the result
print(hourly_aggregated)

                     hourly  call_success  call_fail
0 2024-08-08 03:00:00+00:00             1          0
1 2024-08-08 04:00:00+00:00             1          0
2 2024-08-08 05:00:00+00:00             5          0
3 2024-08-08 06:00:00+00:00             2          0
4 2024-08-09 04:00:00+00:00             2          0


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_48268\1498039094.py:1: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [ ]:
op_consumer['hourly'] = op_consumer['timestamp'].dt.floor('H')

# Step 2: Group by the hourly timestamp and sum the success counts
op_hourly_aggregated = op_consumer.groupby('hourly')[['call_success','call_fail']].sum().reset_index()

# Display the result
print(op_hourly_aggregated)

                     hourly  call_success  call_fail
0 2024-08-06 22:00:00+00:00             3          0
1 2024-08-06 23:00:00+00:00            16          0
2 2024-08-08 06:00:00+00:00             2          0
3 2024-08-08 21:00:00+00:00             4          0
4 2024-08-08 23:00:00+00:00             2          0


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_48268\3394169483.py:1: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [ ]:
node['hourly'] = node['timestamp'].dt.floor('H')

# Step 2: Group by the hourly timestamp and sum the success counts
node_hourly_aggregated = node.groupby('hourly')[['call_success','call_fail']].sum().reset_index()

# Display the result
print(node_hourly_aggregated)

                     hourly  call_success  call_fail
0 2024-08-06 22:00:00+00:00             3          0
1 2024-08-06 23:00:00+00:00            16          0
2 2024-08-08 06:00:00+00:00             2          0
3 2024-08-08 21:00:00+00:00             4          0
4 2024-08-08 23:00:00+00:00             2          0


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_48268\3394169483.py:1: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [ ]:
import pandas as pd

# Assuming hourly_aggregated is the DataFrame we created earlier
# Create a complete range of hourly timestamps
full_range = pd.date_range(start=hourly_aggregated['hourly'].min(), 
                           end=hourly_aggregated['hourly'].max(), 
                           freq='H')

# Convert to DataFrame
full_range_df = pd.DataFrame(full_range, columns=['hourly'])

# Merge the full range with the aggregated data
hourly_filled = pd.merge(full_range_df, hourly_aggregated, on='hourly', how='left')

# Fill missing success counts with 0
hourly_filled['call_success'] = hourly_filled['call_success'].fillna(0)
hourly_filled['call_fail'] = hourly_filled['call_fail'].fillna(0)


# Display the result
print(hourly_filled)


                      hourly  call_success  call_fail
0  2024-08-08 03:00:00+00:00           1.0        0.0
1  2024-08-08 04:00:00+00:00           1.0        0.0
2  2024-08-08 05:00:00+00:00           5.0        0.0
3  2024-08-08 06:00:00+00:00           2.0        0.0
4  2024-08-08 07:00:00+00:00           0.0        0.0
5  2024-08-08 08:00:00+00:00           0.0        0.0
6  2024-08-08 09:00:00+00:00           0.0        0.0
7  2024-08-08 10:00:00+00:00           0.0        0.0
8  2024-08-08 11:00:00+00:00           0.0        0.0
9  2024-08-08 12:00:00+00:00           0.0        0.0
10 2024-08-08 13:00:00+00:00           0.0        0.0
11 2024-08-08 14:00:00+00:00           0.0        0.0
12 2024-08-08 15:00:00+00:00           0.0        0.0
13 2024-08-08 16:00:00+00:00           0.0        0.0
14 2024-08-08 17:00:00+00:00           0.0        0.0
15 2024-08-08 18:00:00+00:00           0.0        0.0
16 2024-08-08 19:00:00+00:00           0.0        0.0
17 2024-08-08 20:00:00+00:00

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_48268\1280086061.py:5: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [ ]:
import pandas as pd

# Assuming hourly_aggregated is the DataFrame we created earlier
# Create a complete range of hourly timestamps
op_full_range = pd.date_range(start=op_hourly_aggregated['hourly'].min(), 
                           end=op_hourly_aggregated['hourly'].max(), 
                           freq='H')

# Convert to DataFrame
op_full_range_df = pd.DataFrame(op_full_range, columns=['hourly'])

# Merge the full range with the aggregated data
op_hourly_filled = pd.merge(op_full_range_df, op_hourly_aggregated, on='hourly', how='left')

# Fill missing success counts with 0
op_hourly_filled['call_success'] = op_hourly_filled['call_success'].fillna(0)
op_hourly_filled['call_fail'] = op_hourly_filled['call_fail'].fillna(0)


# Display the result
print(op_hourly_filled)


                      hourly  call_success  call_fail
0  2024-08-06 22:00:00+00:00           3.0        0.0
1  2024-08-06 23:00:00+00:00          16.0        0.0
2  2024-08-07 00:00:00+00:00           0.0        0.0
3  2024-08-07 01:00:00+00:00           0.0        0.0
4  2024-08-07 02:00:00+00:00           0.0        0.0
5  2024-08-07 03:00:00+00:00           0.0        0.0
6  2024-08-07 04:00:00+00:00           0.0        0.0
7  2024-08-07 05:00:00+00:00           0.0        0.0
8  2024-08-07 06:00:00+00:00           0.0        0.0
9  2024-08-07 07:00:00+00:00           0.0        0.0
10 2024-08-07 08:00:00+00:00           0.0        0.0
11 2024-08-07 09:00:00+00:00           0.0        0.0
12 2024-08-07 10:00:00+00:00           0.0        0.0
13 2024-08-07 11:00:00+00:00           0.0        0.0
14 2024-08-07 12:00:00+00:00           0.0        0.0
15 2024-08-07 13:00:00+00:00           0.0        0.0
16 2024-08-07 14:00:00+00:00           0.0        0.0
17 2024-08-07 15:00:00+00:00

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_48268\2781388024.py:5: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [ ]:
import pandas as pd

# Assuming hourly_aggregated is the DataFrame we created earlier
# Create a complete range of hourly timestamps
node_full_range = pd.date_range(start=node_hourly_aggregated['hourly'].min(), 
                           end=node_hourly_aggregated['hourly'].max(), 
                           freq='H')

# Convert to DataFrame
node_full_range_df = pd.DataFrame(node_full_range, columns=['hourly'])

# Merge the full range with the aggregated data
node_hourly_filled = pd.merge(node_full_range_df, node_hourly_aggregated, on='hourly', how='left')

# Fill missing success counts with 0
node_hourly_filled['call_success'] = node_hourly_filled['call_success'].fillna(0)
node_hourly_filled['call_fail'] = node_hourly_filled['call_fail'].fillna(0)


# Display the result
print(node_hourly_filled)


NameError: name 'node_hourly_aggregated' is not defined

In [ ]:
base_sepolia_chart = make_subplots(specs=[[{"secondary_y": True}]])
        
base_sepolia_chart.add_trace(
    go.Bar(
        x=hourly_filled['hourly'],
        y=hourly_filled['call_success'],
        name='Successful Calls',
    ),
    secondary_y=False
)

base_sepolia_chart.add_trace(
    go.Bar(
        x=hourly_filled['hourly'],
        y=hourly_filled['call_fail'],
        name='Failed Calls',
    ),
    secondary_y=False
)

base_sepolia_chart.update_layout(
    barmode='stack',
    legend=dict(yanchor="top", y=1.1, xanchor="left", x=0.01,orientation="h",bgcolor='rgba(0,0,0,0)'
), plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)')
base_sepolia_chart.update_layout(
    title='Base-Sepolia Chainlink Calls per Hour',
    # barmode='group'  # Set the bar mode to either 'group' for side-by-side or 'stack' for stacked
)

base_sepolia_chart.show()

In [ ]:
op_sepolia_chart = make_subplots(specs=[[{"secondary_y": True}]])
        
op_sepolia_chart.add_trace(
    go.Bar(
        x=op_hourly_filled['hourly'],
        y=op_hourly_filled['call_success'],
        name='Successful Calls',
    ),
    secondary_y=False
)

op_sepolia_chart.add_trace(
    go.Bar(
        x=op_hourly_filled['hourly'],
        y=op_hourly_filled['call_fail'],
        name='Failed Calls',
    ),
    secondary_y=False
)

op_sepolia_chart.update_layout(
    barmode='stack',
    legend=dict(yanchor="top", y=1.1, xanchor="left", x=0.01,orientation="h",bgcolor='rgba(0,0,0,0)'
), plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)')
op_sepolia_chart.update_layout(
    title='Optimism-Sepolia Chainlink Calls per Hour',
    # barmode='group'  # Set the bar mode to either 'group' for side-by-side or 'stack' for stacked
)

op_sepolia_chart.show()

In [ ]:
node_sepolia_chart = make_subplots(specs=[[{"secondary_y": True}]])
        
node_sepolia_chart.add_trace(
    go.Bar(
        x=node_hourly_filled['hourly'],
        y=node_hourly_filled['call_success'],
        name='Successful Calls',
    ),
    secondary_y=False
)

node_sepolia_chart.add_trace(
    go.Bar(
        x=node_hourly_filled['hourly'],
        y=node_hourly_filled['call_fail'],
        name='Failed Calls',
    ),
    secondary_y=False
)

node_sepolia_chart.update_layout(
    barmode='stack',
    legend=dict(yanchor="top", y=1.1, xanchor="left", x=0.01,orientation="h",bgcolor='rgba(0,0,0,0)'
))
node_sepolia_chart.update_layout(
    title='Chainlink Node Calls per Hour',
    # barmode='group'  # Set the bar mode to either 'group' for side-by-side or 'stack' for stacked
)

node_sepolia_chart.show()

NameError: name 'node_hourly_filled' is not defined